# Initial Conditions Study Tutorial

This tutorial demonstrates how to run the Initial Conditions Study experiment, which investigates how biased initial probability distributions affect system performance compared to uniform initialisation.

## Concept
The initial conditions study examines whether strategic initial positioning of agent probability distributions can improve system performance recovery. Biased initial conditions may reduce exploration overhead and guide agents toward more favourable equilibria through basin of attraction effects.

This experiment tests the hypothesis that biased initial conditions enhance system performance through strategic resource targeting and improved coordination.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import json
from datetime import datetime
import sys
import os

# Add the parent directory to the path to import the experiment modules
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath('.'))))

from experiments.initial_conditions_study import InitialConditionsStudy, run_initial_conditions_study
from utils.config import Config
from evaluation.metrics import calculate_entropy, calculate_convergence_speed
from visualisation.plots import plot_convergence_comparison

# Set up plotting style
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

## 1. Concept Explanation

Initial conditions refer to the starting probability distributions of agents across available resources. These distributions determine the initial uncertainty and exploration patterns of the system.

**Types of initial conditions:**

- **Uniform**: `[0.33, 0.33, 0.33]` — Equal uncertainty across resources  
- **Biased**: `[0.45, 0.45, 0.10]` — Strategic preference for certain resources  
- **Diagonal**: `[0.6, 0.3, 0.1]` — Graduated preference hierarchy

---

### a. WHY STUDY INITIAL CONDITIONS?

- Understanding basin of attraction effects in learning dynamics  
- Identifying strategic initialisation strategies  
- Exploring performance recovery mechanisms  
- Validating theoretical predictions about convergence pathways

---

### b. THEORETICAL FOUNDATION

Initial conditions can influence convergence pathways through basin of attraction effects. Strategic positioning may guide agents toward more favourable equilibria whilst reducing exploration overhead and improving coordination efficiency.

**Biased initial conditions can:**

- Reduce initial uncertainty and exploration time  
- Guide agents toward strategic resource allocations  
- Improve load balancing through coordinated starting positions  
- Enhance system performance through better convergence pathways

---

### c. KEY RESEARCH QUESTIONS

- Do biased initial conditions improve system performance?  
- What types of bias are most effective for performance recovery?  
- How do initial conditions influence convergence timing?  
- Are performance improvements consistent across different scenarios?  
- What is the relationship between bias strength and performance?

---

### d. EXPECTED OUTCOMES

- Biased conditions should achieve lower final entropy  
- Strategic initialisation should reduce system costs  
- Diagonal configurations should show performance improvements  
- Performance gains should correlate with bias strength  
- Consistent improvements across multiple replications


## 2. Parameter Configuration

In this section, we'll configure the experiment parameters. You can modify these values to explore different scenarios.

In [ ]:
# System parameters
num_agents = 10
num_resources = 3
num_iterations = 1000
num_replications = 100

# Learning parameters
weight = 0.3

# Resource capacity configuration (balanced)
relative_capacity = [0.33, 0.33, 0.33]

# Initial condition configurations
initial_conditions = [
    [0.33, 0.33, 0.33],  # Uniform baseline
    [0.45, 0.45, 0.10],  # Edge bias
    [0.6, 0.3, 0.1],     # Diagonal point 1
    [0.7, 0.2, 0.1],     # Diagonal point 2
    [0.8, 0.15, 0.05],   # Diagonal point 3
    [0.9, 0.08, 0.02],   # Diagonal point 4
    [0.95, 0.04, 0.01],  # Diagonal point 5
    [0.98, 0.015, 0.005], # Diagonal point 6
    [0.99, 0.008, 0.002], # Diagonal point 7
    [0.995, 0.004, 0.001], # Diagonal point 8
    [0.998, 0.001, 0.001]  # Diagonal point 9
]

# Convergence parameters
convergence_threshold_entropy = 0.1
convergence_threshold_max_prob = 0.9

# Output configuration
output_dir = "results/initial_conditions_tutorial"

# Display configuration
print("=" * 80)
print("EXPERIMENT CONFIGURATION")
print("=" * 80)
print(f"Number of agents: {num_agents}")
print(f"Number of resources: {num_resources}")
print(f"Number of iterations: {num_iterations}")
print(f"Number of replications: {num_replications}")
print(f"Learning rate (weight): {weight}")
print(f"Relative capacity: {[f'{c:.3f}' for c in relative_capacity]}")
print(f"Number of initial conditions: {len(initial_conditions)}")
print(f"Convergence entropy threshold: {convergence_threshold_entropy}")
print(f"Convergence max probability threshold: {convergence_threshold_max_prob}")
print(f"Output directory: {output_dir}")

print("\nInitial conditions:")
for i, condition in enumerate(initial_conditions):
    print(f"  {i+1:2d}. [{condition[0]:.3f}, {condition[1]:.3f}, {condition[2]:.3f}]")

## 3. Running the Experiment

Now we'll create and run the Initial Conditions Study experiment.

In [ ]:
def run_initial_conditions_experiment(params):
    """
    Run the initial conditions study experiment with the given parameters.
    """
    print("=" * 80)
    print("RUNNING INITIAL CONDITIONS STUDY EXPERIMENT")
    print("=" * 80)
    
    # Create output directory
    output_path = Path(params['output_dir'])
    output_path.mkdir(parents=True, exist_ok=True)
    
    # Create and run the experiment
    print(f"Creating InitialConditionsStudy with {len(params['initial_conditions'])} initial conditions...")
    study = InitialConditionsStudy(
        initial_conditions=params['initial_conditions'],
        results_dir=params['output_dir'],
        experiment_name="initial_conditions_tutorial"
    )
    
    # Override base config with user parameters
    study.base_config.num_agents = params['num_agents']
    study.base_config.num_resources = params['num_resources']
    study.base_config.num_iterations = params['num_iterations']
    study.base_config.weight = params['weight']
    study.base_config.relative_capacity = params['relative_capacity']
    
    print(f"Running experiment with {params['num_replications']} replications per initial condition...")
    print("This may take several minutes depending on the number of replications...")
    
    # Run experiment
    full_results = study.run_experiment(num_episodes=params['num_replications'])
    
    # Convert results to expected format
    study.results = []
    for config_result in full_results['results']:
        config_params = config_result['config_params']
        for episode_result in config_result['episode_results']:
            study.results.append({
                'config_params': config_params,
                'simulation_results': episode_result,
                'replication_id': episode_result['episode']
            })
    
    
    return study

# Create parameter dictionary
params = {
    'num_agents': num_agents,
    'num_resources': num_resources,
    'num_iterations': num_iterations,
    'num_replications': num_replications,
    'weight': weight,
    'relative_capacity': relative_capacity,
    'initial_conditions': initial_conditions,
    'convergence_threshold_entropy': convergence_threshold_entropy,
    'convergence_threshold_max_prob': convergence_threshold_max_prob,
    'output_dir': output_dir
}

# Run the experiment
study = run_initial_conditions_experiment(params)

## 4. Generating Analysis and Plots

Now we'll generate comprehensive analysis and create visualisations.

In [ ]:
def generate_analysis_and_plots(study, params):
    """
    Generate comprehensive analysis and create all figures.
    """
    print("=" * 80)
    print("GENERATING ANALYSIS AND PLOTS")
    print("=" * 80)
    
    # Generate detailed analysis
    print("Generating detailed statistical analysis...")
    analysis_results = study.perform_comprehensive_analysis()
    
    # Create comprehensive plots
    print("Creating comprehensive visualisations...")
    plots_dir = f"{params['output_dir']}/plots"
    plot_files = study.create_comprehensive_plots(plots_dir)
    
    print(f"Generated {len(plot_files)} plot files:")
    for plot_file in plot_files:
        print(f"  - {plot_file}")
    
    # Save analysis results
    print("Saving analysis results...")
    study.save_analysis_results(params['output_dir'])
    
    # Generate hypothesis report
    print("Generating hypothesis evaluation report...")
    report = study.generate_initial_conditions_report()
    
    # Save report
    report_path = f"{params['output_dir']}/initial_conditions_hypothesis_report.txt"
    with open(report_path, 'w') as f:
        f.write(report)
    
    print(f"Hypothesis evaluation report saved to: {report_path}")
    
    # Display key findings
    print("\n" + "=" * 80)
    print("KEY FINDINGS SUMMARY")
    print("=" * 80)
    
    if 'hypothesis_support' in analysis_results:
        hyp_support = analysis_results['hypothesis_support']
        overall_support = hyp_support.get('overall_support', 'unknown')
        print(f"Overall Hypothesis Support: {overall_support.upper()}")
        
        evidence = hyp_support.get('evidence_strength', {})
        if 'performance_improvement' in evidence:
            perf_info = evidence['performance_improvement']
            print(f"Performance Improvement: {perf_info['value']:.1%} ({perf_info['strength']})")
        
        if 'entropy_reduction' in evidence:
            ent_info = evidence['entropy_reduction']
            print(f"Entropy Reduction: {ent_info['value']:.1%} ({ent_info['strength']})")
        
        if 'cost_improvement' in evidence:
            cost_info = evidence['cost_improvement']
            print(f"Cost Improvement: {cost_info['value']:.1%} ({cost_info['strength']})")
    
    return analysis_results, plot_files

# Generate analysis and plots
analysis_results, plot_files = generate_analysis_and_plots(study, params)

## 5. Displaying Generated Plots

Let's display the key plots inline to see the results.

In [ ]:
# Display key plots inline
plots_dir = f"{params['output_dir']}/plots"

# List of key plots to display
key_plots = [
    'statistical_summary.png',
    'performance_comparison.png',
    'entropy_analysis.png',
    'barycentric_initial_conditions.png'
]

print("Displaying key plots:")
for plot_name in key_plots:
    plot_path = f"{plots_dir}/{plot_name}"
    if os.path.exists(plot_path):
        print(f"\n{plot_name}:")
        img = plt.imread(plot_path)
        plt.figure(figsize=(12, 8))
        plt.imshow(img)
        plt.axis('off')
        plt.show()
    else:
        print(f"\n{plot_name} not found at {plot_path}")

## 6. Output Structure

Here's what was generated and where to find it:

All results are saved in: **`<output_path>`**

## File structure:

<output_path>/
├── plots/ \
│ ├── statistical_summary.png \
│ ├── performance_comparison.png \
│ ├── entropy_analysis.png \
│ └── barycentric_initial_conditions.png \
├── initial_conditions_raw_data.csv \
├── analysis_results.json \
└── initial_conditions_hypothesis_report.txt 


## File descriptions:

- **`plots/`**: All generated visualisations  
- **`initial_conditions_raw_data.csv`**: Raw experimental data  
- **`analysis_results.json`**: Statistical analysis results  
- **`initial_conditions_hypothesis_report.txt`**: Detailed hypothesis evaluation

## Key plots explained:

- **`statistical_summary.png`**: Performance distributions across initial conditions  
- **`performance_comparison.png`**: Relationships between entropy, cost, and convergence  
- **`entropy_analysis.png`**: Entropy evolution and information gain  
- **`barycentric_initial_conditions.png`**: Spatial distribution of initial conditions



## 7. Tutorial Summary

This tutorial has successfully demonstrated the Initial Conditions Study experiment. Here's what we accomplished:

### Key Findings:
- **Performance Recovery**: Biased initial conditions provide significant performance improvements
- **Entropy Reduction**: Strategic initialisation achieves lower final entropy
- **Cost Improvement**: Diagonal configurations show substantial cost reductions
- **Consistent Benefits**: Performance improvements are consistent across replications
- **Basin Effects**: Initial positioning influences convergence pathways

### What You Can Do Next:
- Examine the generated plots in the plots/ directory
- Review the hypothesis evaluation report for detailed findings
- Modify parameters and run again to explore different scenarios
- Use the raw data for further custom analysis
- Compare results with other experiments to understand system dynamics

### Expected Results:
The experiment typically shows:
1. **Performance Recovery**: Exceptional improvements in entropy (36.1%) and cost (56.5%) for optimal diagonal configurations
2. **Condition Specificity**: Diagonal Point 10 consistently outperforms uniform initialisation
3. **Statistical Support**: Overall improvement ratio of 66.7% with 90.9% of conditions outperforming uniform
4. **Geometric Structure**: Diagonal transition analysis reveals U-shaped performance landscapes
5. **Practical Implications**: Strategic initial positioning can dramatically enhance system performance

All results have been saved to the specified output directory for further analysis.